<a href="https://colab.research.google.com/github/OliverWarwick/ThirdYearProject/blob/master/LoadingPetaData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My\ Drive/Datasets

/content/gdrive/My Drive/Datasets


In [117]:
import numpy as np
import re
import time

dataFile = open("Label.txt","r")
imageArr = []
t = time.time()
for line in dataFile:
    # Create image object.
    im = Image()

    imageNumber = int(line[:3]) # Get the image number.
    im.ID = imageNumber

    # Split the string by spaces 
    attributeString = line[4:]
    dataLabels = attributeString.split(" ")

    for label in dataLabels: # Try to find in the dictionary, if not then just print a flag.
        label = re.sub('[\W_]+', '', label) # Get rid of any extra space, line characters etc.

        if label in labelToNumberFull:
            indexOfAttirbute = labelToNumberFull[label] # If normal label then add straight away to singleClassData numpy array.
            im.singleClassData[indexOfAttirbute-1] = 1

        else:
            # For the multiclass labels.
            foundCorrect = False
            listOfMultiClassAttirbutes = ["upperBody","lowerBody","hair","footwear"]

            for word in listOfMultiClassAttirbutes:
                if label[0] == word[0]:  # As all distinct first letters.
                    colour = label.replace(word,'') # Get just the colour
                    try:
                        (im.multiClassData[word])[colourToNumberFull[colour]-1] = 1 # Change the correct index.
                        foundCorrect = True
                    except Exception:
                        print("Unknown Colour: " + colour)
            if not foundCorrect:
                print("Unknown Label: " + label)

    # Finish processing.
    im.concatArrays()
    imageArr.append(im)
    getLowerBodyFeaturesVector(im)

print(time.time()-t)

   
        

[0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0]
[0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0]
[0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1]
[0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1]
[0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1]
[0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0]
[0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1]
[0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1

In [0]:
class Image:
    def __init__(self):
        self.ID = 0

        self.singleClassData = np.zeros((65,), dtype=int)
        self.multiClassData = {"upperBody": np.zeros((11,), dtype=int), "lowerBody": np.zeros((11,), dtype=int),
                               "footwear": np.zeros((11,), dtype=int), "hair": np.zeros((11,), dtype=int)}
        self.fullData = np.zeros((109,), dtype=int)

        self.lowerBodyVector = np.zeros((46,),dtype=int)

    def __str__(self):
        return "Single Class Data:\n{0} \nMulti Class Data: \n{1} \n".format(self.singleClassData,self.multiClassData) # To string method

    def concatArrays(self):
        self.fullData = np.concatenate((self.singleClassData, self.multiClassData["upperBody"], self.multiClassData["lowerBody"], 
                                        self.multiClassData["footwear"], self.multiClassData["hair"]))
        
        
    


All the labels from the readme.txt file in the PETA dataset.

In [0]:
listOfSingleClassData = [(2,"accessoryHeadphone"), (4,"personalLess15"), (5,"personalLess30"), (6,"personalLess45"), (7,"personalLess60"), (8,"personalLarger60"), 
              (9,"carryingBabyBuggy"), (10,"carryingBackpack"), (11, "hairBald"), (12,"footwearBoots"), (13,"lowerBodyCapri"), (14,"carryingOther"), (15,"carryingShoppingTro"),
              (16,"carryingUmbrella"), (17,"lowerBodyCasual"), (18,"upperBodyCasual"), (19,"personalFemale"), (20,"carryingFolder"), (21,"lowerBodyFormal"), 
              (22,"upperBodyFormal"), (23,"accessoryHairBand"), (24,"accessoryHat"), (25, "lowerBodyHotPants"), (26,"upperBodyJacket"), (27,"lowerBodyJeans"),
              (28,"accessoryKerchief"), (29,"footwearLeatherShoes"), (30,"upperBodyLogo"), (31,"hairLong"), (32,"lowerBodyLongSkirt"), (33,"upperBodyLongSleeve"),
              (35,"lowerBodyPlaid"), (37,"lowerBodyThinStripes"), (38,"carryingLuggageCase"), (39,"personalMale"), (40,"carryingMessengerBag"), (41,"accessoryMuffler"),
              (42,"accessoryNothing"), (43,"carryingNothing"), (44,"upperBodyNoSleeve"), (45,"upperBodyPlaid"), (46,"carryingPlasticBags"), (47,"footwearSandals"),
              (48,"footwearShoes"), (49,"hairShort"), (50,"lowerBodyShorts"), (51,"upperBodyShortSleeve"), (52,"lowerBodyShortSkirt"), (53,"footwearSneakers"),
              (54,"footwearStocking"), (55,"upperBodyThinStripes"), (56,"upperBodySuit"), (57,"carryingSuitcase"), (58,"lowerBodySuits"), (59,"accessorySunglasses"),
              (60,"upperBodySweater"), (61,"upperBodyThickStripes"), (62,"lowerBodyTrousers"), (63,"upperBodyTshirt"), (64,"upperBodyOther"), (65,"upperBodyVNeck")]



numberToLabelFull = dict()
for (k,v) in listOfSingleClassData:
    numberToLabelFull[k] = v
print(numberToLabelFull)

labelToNumberFull = dict()
for (k,v) in listOfSingleClassData:
    labelToNumberFull[v] = k  

colourForMultiClassData = [(1,"Black"), (2,"Blue"), (3,"Brown"), (4,"Green"), (5,"Grey"), (6,"Orange"), (7,"Pink"), (8,"Purple"), (9,"Red"), (10,"White"), (11,"Yellow")]

colourToNumberFull = dict()
for (k,v) in colourForMultiClassData:
    colourToNumberFull[k] = v

numberToColourFull = dict()
for (k,v) in colourForMultiClassData:
    colourToNumberFull[v] = k

	

fullAnnotationsList = [(1,"Blank"),(2,"accessoryHeadphone"),(3,"Blank"), (4,"personalLess15"), (5,"personalLess30"), (6,"personalLess45"), (7,"personalLess60"), (8,"personalLarger60"), 
              (9,"carryingBabyBuggy"), (10,"carryingBackpack"), (11, "hairBald"), (12,"footwearBoots"), (13,"lowerBodyCapri"), (14,"carryingOther"), (15,"carryingShoppingTro"),
              (16,"carryingUmbrella"), (17,"lowerBodyCasual"), (18,"upperBodyCasual"), (19,"personalFemale"), (20,"carryingFolder"), (21,"lowerBodyFormal"), 
              (22,"upperBodyFormal"), (23,"accessoryHairBand"), (24,"accessoryHat"), (25, "lowerBodyHotPants"), (26,"upperBodyJacket"), (27,"lowerBodyJeans"),
              (28,"accessoryKerchief"), (29,"footwearLeatherShoes"), (30,"upperBodyLogo"), (31,"hairLong"), (32,"lowerBodyLongSkirt"), (33,"upperBodyLongSleeve"),
              (34,"Blank"), (35,"lowerBodyPlaid"), (36,"Blank"), (37,"lowerBodyThinStripes"), (38,"carryingLuggageCase"), (39,"personalMale"), (40,"carryingMessengerBag"), (41,"accessoryMuffler"),
              (42,"accessoryNothing"), (43,"carryingNothing"), (44,"upperBodyNoSleeve"), (45,"upperBodyPlaid"), (46,"carryingPlasticBags"), (47,"footwearSandals"),
              (48,"footwearShoes"), (49,"hairShort"), (50,"lowerBodyShorts"), (51,"upperBodyShortSleeve"), (52,"lowerBodyShortSkirt"), (53,"footwearSneakers"),
              (54,"footwearStocking"), (55,"upperBodyThinStripes"), (56,"upperBodySuit"), (57,"carryingSuitcase"), (58,"lowerBodySuits"), (59,"accessorySunglasses"),
              (60,"upperBodySweater"), (61,"upperBodyThickStripes"), (62,"lowerBodyTrousers"), (63,"upperBodyTshirt"), (64,"upperBodyOther"), (65,"upperBodyVNeck"), 
              (66,"upperBodyBlack"),(67,"upperBodyBlue"),(68,"upperBodyBrown"),(69,"upperBodyGreen"),(70,"upperBodyGrey"),(71,"upperBodyOrange"),(72,"upperBodyPink"),
              (73,"upperBodyPurple"),(74,"upperBodyRed"),(75,"upperBodyWhite"),(76,"upperBodyYellow"), (77,"lowerBodyBlack"),(78,"lowerBodyBlue"),(79,"lowerBodyBrown"),
              (80,"lowerBodyGreen"),(81,"lowerBodyGrey"),(82,"lowerBodyOrange"),(83,"lowerBodyPink"),(84,"lowerBodyPurple"),(85,"lowerBodyRed"),(86,"lowerBodyWhite"),
              (87,"lowerBodyYellow"),(88,"footwearBlack"),(89,"footwearBlue"),(90,"footwearBrown"),(91,"footwearGreen"),(92,"footwearGrey"),(93,"footwearOrange"),
              (94,"footwearPink"),(95,"footwearPurple"),(96,"footwearRed"),(97,"footwearWhite"),(98,"footwearYellow"),(99,"hairBlack"),(100,"hairBlue"),(101,"hairBrown"),
              (102,"hairGreen"),(103,"hairGrey"),(104,"hairOrange"),(105,"hairPink"),(106,"hairPurple"),(107,"hairRed"),(108,"hairWhite"),(109,"hairYellow")]
	
# Need to fix^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    




Input: Image Object, which has already been initialised.

In [113]:
def getLowerBodyFeaturesVector(image):

    fullAnnotationsList = [(1,"Blank"),(2,"accessoryHeadphone"),(3,"Blank"), (4,"personalLess15"), (5,"personalLess30"), (6,"personalLess45"), (7,"personalLess60"), (8,"personalLarger60"), (9,"carryingBabyBuggy"), (10,"carryingBackpack"), (11, "hairBald"), (12,"footwearBoots"), (13,"lowerBodyCapri"), (14,"carryingOther"), (15,"carryingShoppingTro"),(16,"carryingUmbrella"), (17,"lowerBodyCasual"), (18,"upperBodyCasual"), (19,"personalFemale"), (20,"carryingFolder"), (21,"lowerBodyFormal"), (22,"upperBodyFormal"), (23,"accessoryHairBand"), (24,"accessoryHat"), (25, "lowerBodyHotPants"), (26,"upperBodyJacket"), (27,"lowerBodyJeans"),(28,"accessoryKerchief"), (29,"footwearLeatherShoes"), (30,"upperBodyLogo"), (31,"hairLong"), (32,"lowerBodyLongSkirt"), (33,"upperBodyLongSleeve"),(34,"Blank"), (35,"lowerBodyPlaid"), (36,"Blank"), (37,"lowerBodyThinStripes"), (38,"carryingLuggageCase"), (39,"personalMale"), (40,"carryingMessengerBag"), (41,"accessoryMuffler"),(42,"accessoryNothing"), (43,"carryingNothing"), (44,"upperBodyNoSleeve"), (45,"upperBodyPlaid"), (46,"carryingPlasticBags"), (47,"footwearSandals"),(48,"footwearShoes"), (49,"hairShort"), (50,"lowerBodyShorts"), (51,"upperBodyShortSleeve"), (52,"lowerBodyShortSkirt"), (53,"footwearSneakers"),(54,"footwearStocking"), (55,"upperBodyThinStripes"), (56,"upperBodySuit"), (57,"carryingSuitcase"), (58,"lowerBodySuits"), (59,"accessorySunglasses"),(60,"upperBodySweater"), (61,"upperBodyThickStripes"), (62,"lowerBodyTrousers"), (63,"upperBodyTshirt"), (64,"upperBodyOther"), (65,"upperBodyVNeck"), (66,"upperBodyBlack"),(67,"upperBodyBlue"),(68,"upperBodyBrown"),(69,"upperBodyGreen"),(70,"upperBodyGrey"),(71,"upperBodyOrange"),(72,"upperBodyPink"),(73,"upperBodyPurple"),(74,"upperBodyRed"),(75,"upperBodyWhite"),(76,"upperBodyYellow"), (77,"lowerBodyBlack"),(78,"lowerBodyBlue"),(79,"lowerBodyBrown"),(80,"lowerBodyGreen"),(81,"lowerBodyGrey"),(82,"lowerBodyOrange"),(83,"lowerBodyPink"),(84,"lowerBodyPurple"),(85,"lowerBodyRed"),(86,"lowerBodyWhite"),(87,"lowerBodyYellow"),(88,"footwearBlack"),(89,"footwearBlue"),(90,"footwearBrown"),(91,"footwearGreen"),(92,"footwearGrey"),(93,"footwearOrange"),(94,"footwearPink"),(95,"footwearPurple"),(96,"footwearRed"),(97,"footwearWhite"),(98,"footwearYellow"),(99,"hairBlack"),(100,"hairBlue"),(101,"hairBrown"),(102,"hairGreen"),(103,"hairGrey"),(104,"hairOrange"),(105,"hairPink"),(106,"hairPurple"),(107,"hairRed"),(108,"hairWhite"),(109,"hairYellow")]
	
    # These will look like:
    # list = ((1,"footwearBoots"), (2,"lowerBodyCapri") etc...)
    # Turns out to be a 46 
    # Want to isolate the features which are important to the lowerhalf of the body.
    # All of these indicies will be one above the machine implimentation of them.

    # Order will not be the same as above.

    lowerBodyIndices = [12,13,16,17,19,21,25,27,29,32,35,37,38,39,40,43,47,48,50,52,53,54,57,58,62] + list(range(77,98))
    tmpVector = np.zeros((46,),dtype=int)

    for i in range(0,len(lowerBodyIndices)):
        image.lowerBodyVector[i] = image.fullData[lowerBodyIndices[i] - 1]

    print(image.lowerBodyVector)

    # Get the array we need.

    
def getLowerBodyFeatureAttirbuteNames():
    fullAnnotationsList = [(1,"Blank"),(2,"accessoryHeadphone"),(3,"Blank"), (4,"personalLess15"), (5,"personalLess30"), (6,"personalLess45"), (7,"personalLess60"), (8,"personalLarger60"), (9,"carryingBabyBuggy"), (10,"carryingBackpack"), (11, "hairBald"), (12,"footwearBoots"), (13,"lowerBodyCapri"), (14,"carryingOther"), (15,"carryingShoppingTro"),(16,"carryingUmbrella"), (17,"lowerBodyCasual"), (18,"upperBodyCasual"), (19,"personalFemale"), (20,"carryingFolder"), (21,"lowerBodyFormal"), (22,"upperBodyFormal"), (23,"accessoryHairBand"), (24,"accessoryHat"), (25, "lowerBodyHotPants"), (26,"upperBodyJacket"), (27,"lowerBodyJeans"),(28,"accessoryKerchief"), (29,"footwearLeatherShoes"), (30,"upperBodyLogo"), (31,"hairLong"), (32,"lowerBodyLongSkirt"), (33,"upperBodyLongSleeve"),(34,"Blank"), (35,"lowerBodyPlaid"), (36,"Blank"), (37,"lowerBodyThinStripes"), (38,"carryingLuggageCase"), (39,"personalMale"), (40,"carryingMessengerBag"), (41,"accessoryMuffler"),(42,"accessoryNothing"), (43,"carryingNothing"), (44,"upperBodyNoSleeve"), (45,"upperBodyPlaid"), (46,"carryingPlasticBags"), (47,"footwearSandals"),(48,"footwearShoes"), (49,"hairShort"), (50,"lowerBodyShorts"), (51,"upperBodyShortSleeve"), (52,"lowerBodyShortSkirt"), (53,"footwearSneakers"),(54,"footwearStocking"), (55,"upperBodyThinStripes"), (56,"upperBodySuit"), (57,"carryingSuitcase"), (58,"lowerBodySuits"), (59,"accessorySunglasses"),(60,"upperBodySweater"), (61,"upperBodyThickStripes"), (62,"lowerBodyTrousers"), (63,"upperBodyTshirt"), (64,"upperBodyOther"), (65,"upperBodyVNeck"), (66,"upperBodyBlack"),(67,"upperBodyBlue"),(68,"upperBodyBrown"),(69,"upperBodyGreen"),(70,"upperBodyGrey"),(71,"upperBodyOrange"),(72,"upperBodyPink"),(73,"upperBodyPurple"),(74,"upperBodyRed"),(75,"upperBodyWhite"),(76,"upperBodyYellow"), (77,"lowerBodyBlack"),(78,"lowerBodyBlue"),(79,"lowerBodyBrown"),(80,"lowerBodyGreen"),(81,"lowerBodyGrey"),(82,"lowerBodyOrange"),(83,"lowerBodyPink"),(84,"lowerBodyPurple"),(85,"lowerBodyRed"),(86,"lowerBodyWhite"),(87,"lowerBodyYellow"),(88,"footwearBlack"),(89,"footwearBlue"),(90,"footwearBrown"),(91,"footwearGreen"),(92,"footwearGrey"),(93,"footwearOrange"),(94,"footwearPink"),(95,"footwearPurple"),(96,"footwearRed"),(97,"footwearWhite"),(98,"footwearYellow"),(99,"hairBlack"),(100,"hairBlue"),(101,"hairBrown"),(102,"hairGreen"),(103,"hairGrey"),(104,"hairOrange"),(105,"hairPink"),(106,"hairPurple"),(107,"hairRed"),(108,"hairWhite"),(109,"hairYellow")]
    lowerBodyIndices = [12,13,16,17,19,21,25,27,29,32,35,37,38,39,40,43,47,48,50,52,53,54,57,58,62] + list(range(77,98))
    lowerBodyAttributeNames = []

    count = 1
    for index in lowerBodyIndices:
        lowerBodyAttributeNames.append((count,fullAnnotationsList[index-1][1]))
        count += 1

    return lowerBodyAttributeNames


getLowerBodyFeatureAttirbuteNames()



[(1, 'footwearBoots'),
 (2, 'lowerBodyCapri'),
 (3, 'carryingUmbrella'),
 (4, 'lowerBodyCasual'),
 (5, 'personalFemale'),
 (6, 'lowerBodyFormal'),
 (7, 'lowerBodyHotPants'),
 (8, 'lowerBodyJeans'),
 (9, 'footwearLeatherShoes'),
 (10, 'lowerBodyLongSkirt'),
 (11, 'lowerBodyPlaid'),
 (12, 'lowerBodyThinStripes'),
 (13, 'carryingLuggageCase'),
 (14, 'personalMale'),
 (15, 'carryingMessengerBag'),
 (16, 'carryingNothing'),
 (17, 'footwearSandals'),
 (18, 'footwearShoes'),
 (19, 'lowerBodyShorts'),
 (20, 'lowerBodyShortSkirt'),
 (21, 'footwearSneakers'),
 (22, 'footwearStocking'),
 (23, 'carryingSuitcase'),
 (24, 'lowerBodySuits'),
 (25, 'lowerBodyTrousers'),
 (26, 'lowerBodyBlack'),
 (27, 'lowerBodyBlue'),
 (28, 'lowerBodyBrown'),
 (29, 'lowerBodyGreen'),
 (30, 'lowerBodyGrey'),
 (31, 'lowerBodyOrange'),
 (32, 'lowerBodyPink'),
 (33, 'lowerBodyPurple'),
 (34, 'lowerBodyRed'),
 (35, 'lowerBodyWhite'),
 (36, 'lowerBodyYellow'),
 (37, 'footwearBlack'),
 (38, 'footwearBlue'),
 (39, 'footwearBr